# Play ground for station data

In [45]:
import geozip
import sqlite3 as lite
from geolocation import GeoLocation

In [61]:
def get_distance_to(origin, dest):
    return origin.distance_to(GeoLocation.from_degrees(dest['latitude'], dest['longitude']))

In [95]:
def dict_factory(cursor, row):
    d = {}
    for idx, col in enumerate(cursor.description):
        d[col[0]] = row[idx]
    return d

In [112]:
def fetch_neighbors(lat, lng, con, distance=1):
    origin = GeoLocation.from_degrees(lat, lng)
    bbox = origin.bounding_locations(distance)
    sw = geozip.encode(bbox[0].deg_lat, bbox[0].deg_lon)
    ne = geozip.encode(bbox[1].deg_lat, bbox[1].deg_lon)
    with con:
        cur = con.cursor()
        cur.execute("select * from Stations where geozip > ? and geozip < ?;", (sw,ne))
        stations = cur.fetchall()
    
    neighbors = [dict(x.items() + {"distance":get_distance_to(origin, x)}.items()) for x in stations if get_distance_to(origin, x) <= distance]
    return sorted(neighbors, key=lambda k: k['distance'])

In [119]:
DB = 'stations.db'
con = lite.connect(DB)
con.row_factory = dict_factory

stations = fetch_neighbors(35.6601719, 139.7300166, con, 1)

In [120]:
for s in stations:
    print "%s, %f" % (s['name'], s["distance"])
    

六本木, 0.323306
六本木, 0.440322
麻布十番, 0.686376
乃木坂, 0.791271
麻布十番, 0.881826
